In [1]:
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Enumerator
import Control.Monad.Bayes.Sampler
import Control.Monad.Bayes.Weighted
import Control.Monad.Bayes.Traced
import Control.Monad.Bayes.Population
import Graphics.Vega.VegaLite hiding (density)
import IHaskell.Display.Hvega (vlShow)
import Data.Aeson
import qualified Data.Text as T


:ext OverloadedStrings
:ext OverloadedLists
:e GADTs
:e FlexibleContexts
:e FlexibleInstances
:e DeriveFunctor
:e GeneralizedNewtypeDeriving

In [2]:

import System.Random.MWC (createSystemRandom)
import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Data.Aeson (ToJSON(toJSON), Value)
import Data.Aeson (encode)
import qualified Data.ByteString.Lazy.Char8 as BL
import Data.Text (Text, pack)

import Control.Monad (liftM2, replicateM, forM, forM_, (<=<))
import Control.Monad.IO.Class (liftIO)
import Data.List (sort)
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import Control.Monad.Bayes.Traced
import Control.Monad.Bayes.Weighted
import Control.Monad.Bayes.Inference.SMC as SMC
import Control.Monad.Bayes.Inference.RMSMC as RMSMC
import Control.Monad.Bayes.Sequential
import Control.Monad.Bayes.Population
import Control.Monad.Bayes.Traced.Static (Traced)
import Control.Monad.Bayes.Inference.SMC

import Numeric.Log
import Control.Monad.Bayes.Class

import Data.List (partition)

barPlot :: Text -> VL.VLSpec
barPlot xName = 
    let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Nominal]
            . VL.position VL.Y [VL.PName "binnedData", VL.PAggregate VL.Count, VL.PmType VL.Quantitative, VL.PAxis [VL.AxTitle "count"]]
    in VL.asSpec [VL.mark VL.Bar [VL.MOpacity 1.0, VL.MColor "#a3c6de"], encoding []]

linePlot :: Text -> Text -> VL.VLSpec
linePlot xName yName = 
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative]
  in VL.asSpec [VL.mark VL.Line [VL.MColor "blue"], encoding []]

scatterBlue xName yName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
  in VL.asSpec [VL.mark VL.Circle [VL.MColor "blue"], encoding []]
  
scatterGreen xName yName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
  in VL.asSpec [VL.mark VL.Circle [VL.MColor "green"], encoding []]
  
scatterPlotWithColor :: Text -> Text -> Text -> (Double, Double) -> (Double, Double) -> VL.VLSpec
scatterPlotWithColor xName yName zName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
            . VL.color [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
  in VL.asSpec [VL.mark VL.Circle [], encoding []]

density2DPlot :: Text -> Text -> (Double, Double) -> (Double, Double) -> VL.VLSpec
density2DPlot xName yName (xmin, xmax) (ymin, ymax) = 
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PBin [VL.Nice False, VL.Steps [0.05, 0.5, 5.0], VL.Extent xmin xmax], VL.PmType VL.Quantitative]
            . VL.position VL.Y [VL.PName yName, VL.PBin [VL.Nice False, VL.Steps [0.05, 0.5, 5.0], VL.Extent ymin ymax], VL.PmType VL.Quantitative]
            . VL.color [ VL.MAggregate VL.Count, VL.MName "col", VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
  in VL.asSpec [VL.mark VL.Rect [], encoding []]

imagePlot :: Text -> Text -> Text -> VL.VLSpec
imagePlot xName yName zName =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Nominal, VL.PAxis [VL.AxGridOpacity 0.1]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Nominal, VL.PSort [VL.Descending], VL.PAxis [VL.AxGridOpacity 0.1]]
            . VL.fill [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
            . VL.stroke [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]],
                          VL.MLegend [VL.LType VL.GradientLegend]]
  in VL.asSpec [VL.mark VL.Rect [], encoding []]
  
imageFacetPlot :: Text -> Text -> Text -> VL.VLSpec
imageFacetPlot xName yName zName =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Ordinal, VL.PAxis [VL.AxGrid False]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Ordinal, VL.PSort [VL.Descending], VL.PAxis [VL.AxGrid False]]
            . VL.fill [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]], VL.MLegend [VL.LOrient VL.LOBottom]]
            . VL.stroke [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]],
                          VL.MLegend [VL.LOrient VL.LOBottom, VL.LDirection VL.Horizontal, VL.LType VL.GradientLegend]]
  in VL.asSpec [VL.mark VL.Rect [], encoding [], VL.width 200,  VL.height 100]

data SpecGrid = H [[VL.VLSpec]] | V [[VL.VLSpec]] | L [VL.VLSpec] | S VL.VLSpec | F (Text, Int, VL.VLSpec)

data InputData = Cols [(Text, VL.DataValues)]
               | File FilePath

plot :: (Double, Double) -> SpecGrid -> InputData -> VL.VegaLite
plot (figw,figh) specGrid dataPoints =
    let description = VL.description "Plot"
        dat' = case dataPoints of
            Cols cols -> foldl (.) (VL.dataFromColumns []) (map (uncurry VL.dataColumn) cols) []
            File fp -> VL.dataFromSource (pack fp) []
        configure = VL.configure
            . VL.configuration (VL.Axis
                                        [ VL.Domain False,
                                          VL.LabelColor "#7F7F7F",
                                          VL.LabelPadding 4,
                                          VL.TickColor "#7F7F7F",
                                          VL.TickSize 5.67,
                                          VL.Grid True,
                                          VL.GridColor "#FFFFFF"
                                          ])
        spec = case specGrid of
            S s -> VL.layer [s]
            L ls -> VL.layer ls
            H lss -> VL.hConcat (map (VL.asSpec . (:[]) . VL.layer) lss)
            V lss -> VL.vConcat (map (VL.asSpec . (:[]) . VL.layer) lss)
            F (_, _, s) -> VL.specification s
        facet = case specGrid of
            F (field, nColumns, _) -> [VL.columns $ fromIntegral nColumns, VL.facetFlow [VL.FName field, VL.FmType VL.Nominal]]
            _   -> [VL.width figw,  VL.height figh]
    in VL.toVegaLite $ [VL.background "#f9f9f9", configure [], description, dat', spec] ++ facet

Line 5: Use fewer imports
Found:
import Data.Aeson ( ToJSON(toJSON), Value )
import Data.Aeson ( encode )

Why not:
import Data.Aeson ( ToJSON(toJSON), Value, encode )
Line 12: Use fewer imports
Found:
import Data.List ( sort )
import Data.List ( partition )

Why not:
import Data.List ( sort, partition )
Line 13: Use fewer imports
Found:
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Class

Why not:
import Control.Monad.Bayes.Class
Line 17: Use fewer imports
Found:
import Control.Monad.Bayes.Inference.SMC as SMC
import Control.Monad.Bayes.Inference.SMC

Why not:
import Control.Monad.Bayes.Inference.SMC as SMC

In [3]:

class Plottable a where
    plotVega :: a -> VegaLiteLab

instance Plottable [(Double, Double)] where
    plotVega ds = let (nums, nums2) = unzip ds in vlShow $ plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
        [("x", VL.Numbers nums),
        ("y", VL.Numbers nums2)
        ])

instance Plottable [(Double)] where
    plotVega nums = vlShow $ plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
        [("x", VL.Numbers nums),
        ("y", VL.Numbers (take (length nums) $ Prelude.repeat 0))
        ])

-- instance Plottable [Bool] where
--     plotVega nums = plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
--         [("x", VL.Numbers nums),
--         ("y", VL.Numbers (take (length nums) $ Prelude.repeat 0))
--         ])

-- instance Plottable (Double -> Double) where
--     plotVega f = vlShow $ plot (600, 300)
--               (L [linePlot "x" "y"])
--               (Cols [("x", VL.Numbers $ fromIntegral <$> range), ("y", VL.Numbers (f <$> range))])        

Line 11: Redundant bracket
Found:
(Double)
Why not:
DoubleLine 14: Use replicate
Found:
take (length nums) $ Prelude.repeat 0
Why not:
replicate (length nums) 0

In [4]:
-- vlShow $ plot (200, 100) (L [barPlot "b"]) (Cols [("b", VL.Booleans $ sampleSTfixed $ prior $ mh 10 $ bernoulli 0.5)])

multinorm (a,b) = do
    x <- normal a 1
    y <- normal b 1
    return (x,y)

mixture = do
    x <- uniformD [multinorm (a,b) | (a,b) <- [(1,1), (5,5)]]
    x

(nums, nums2) <- unzip <$> sampleIO (replicateM 1000 $ mixture)
vlShow $ plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
        [("x", VL.Numbers nums),
        ("y", VL.Numbers nums2)
        ])


In [5]:
whichCluster point = do
    let (a',b') = point
    cluster@(a,b) <- uniformD [(1,1), (5,5)]
    factor (normalPdf a 1 a' * normalPdf b 1 b')
    -- prediction <- multinorm cluster
    -- condition (prediction == point )
    return cluster


enumerate $ whichCluster (3,3)
-- sampleIO $ runPopulation $ smcMultinomial 10 10 $ (whichCluster (3,3))

[((1.0,1.0),0.49999999999999994),((5.0,5.0),0.49999999999999994)]

In [6]:
samples <- sampleIO $ prior $ mh 10000 $ whichCluster (4,3)

a = length samples
b = length $ Prelude.filter (==(5,5)) samples


In [7]:
vlShow $ plot (200, 100) (L [barPlot "b"]) (Cols [("b", VL.Booleans $ (take (a-b) $ Prelude.repeat True) <> (take (b) $ Prelude.repeat False))])


Line 1: Move brackets to avoid $
Found:
(take (a - b) $ Prelude.repeat True)
  <> (take (b) $ Prelude.repeat False)
Why not:
(take (a - b) $ Prelude.repeat True)
  <> take (b) (Prelude.repeat False)Line 1: Move brackets to avoid $
Found:
(take (a - b) $ Prelude.repeat True)
  <> (take (b) $ Prelude.repeat False)
Why not:
take (a - b) (Prelude.repeat True)
  <> (take (b) $ Prelude.repeat False)Line 1: Use replicate
Found:
take (a - b) $ Prelude.repeat True
Why not:
replicate (a - b) TrueLine 1: Use replicate
Found:
take (b) $ Prelude.repeat False
Why not:
replicate b FalseLine 1: Redundant bracket
Found:
(b)
Why not:
b

todo: 

get dataset for logistic regression
plot dataset
do a mouseover
plot posterior predictive samples on top

get diagrams working: show in notebook


In [8]:
xs :: [Double]
xs = [-10, -5, 5, 6, 10]

labels :: [Bool]
labels = [False, False, True, True, True]

logisticRegression :: (MonadInfer m) => [(Double, Bool)] -> m [Bool]
logisticRegression dat = do
  m <- normal 0 1
  b <- normal 0 1
  sigma <- gamma 1 1
  let y x = normal (m * x + b) sigma
      sigmoid x = y x >>= \t -> return $ 1 / (1 + exp (- t))
      obs x label = do
        p <- sigmoid x
        factor $ (Exp . log) $ if label then p else 1 - p
  mapM_ (uncurry obs) dat
  mapM (bernoulli <=< sigmoid) $ fmap fst dat

forward m b x = 1 / (1 + exp (- (m * x + b)))

syntheticData :: MonadSample m => Int -> m [(Double, Bool)]
syntheticData n = replicateM n syntheticPoint
  where
    syntheticPoint = do
      x <- uniform (-1) 1
      label <- bernoulli 0.5
      return (x, label)

boolToInt True = 10
boolToInt _ = -10



In [9]:
-- pp <- (sampleIO $ prior $ mh 1000 $ logisticRegression $ zip xs labels)

-- pp

In [10]:
-- vlShow $ plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
--         [("x", VL.Numbers xs),
--         ("y", VL.Numbers (boolToInt <$> labels))
--         ])

-- pp <- (sampleIO $ prior $ mh 1000 $ logisticRegression $ zip xs labels)

-- vlShow $ plot (200, 100) (L [scatterBlue "x" "y" (-10,10) (-10,10)]) (Cols 
--         [("x", VL.Numbers $ take 64 $ cycle [1,2,3,4,5,6,7,8]),
--         ("y", VL.Numbers (take 8 $ pp))
--         ])

In [11]:
samples <- sampleIO $ prior $ mh 10000 $ logisticRegression $ zip xs labels

In [12]:
-- take 10 samples
-- (means, biases) = unzip samples
-- av ls = Prelude.sum ls / (fromIntegral $ Prelude.length ls)
-- av biases

In [13]:
import Control.Monad.Bayes.Population
import Control.Monad.Bayes.Sequential
import Control.Monad.Bayes.Inference.SMC
import Control.Monad.Bayes.Inference.RMSMC
samples <- sampleIOfixed $ runPopulation $ rmsmc 10 10 10 $ logisticRegression $ zip xs labels

In [14]:
-- samples

-- plotVega ((forward (av means) (av biases)) :: (Double -> Double))

In [15]:
range = [0,0.1..10] :: [Double]
samples <- sampleIOfixed $ regressionWithOutliersData range
-- plotVega (zip range samples)

: 

In [ ]:
baseData = (dataFromColumns [ ]
  . dataColumn "X" (Numbers range)
  . dataColumn "Y" (Numbers (fst <$> samples))
  . dataColumn "Outlier" (Strings (T.pack . show . snd <$> samples)))


baseEncoding = encoding
                    . position X [ PName "X" ]
                    . position Y [ PName "Y" ]
                    . color cluster
                    -- . shape cluster

scaleOpts = [ SType ScLog, SDomain (DNumbers [3.5, 32]), SNice (IsNice False) ]
cluster = [ MName "Outlier"]

-- dat = (VL.VLData, either (error . show) id $ A.eitherDecode "['x',  'y']")


showPlot enc dat = vlShow $ toVegaLite [ 
            dat [],
            mark Point []
              , enc []
              , width 200
              , height 200
              ]

showPlot baseEncoding baseData

Line 1: Redundant bracket
Found:
(dataFromColumns []
   . dataColumn "X" (Numbers range)
       . dataColumn "Y" (Numbers (fst <$> samples))
           . dataColumn "Outlier" (Strings (T.pack . show . snd <$> samples)))
Why not:
dataFromColumns []
  . dataColumn "X" (Numbers range)
      . dataColumn "Y" (Numbers (fst <$> samples))
          . dataColumn "Outlier" (Strings (T.pack . show . snd <$> samples))

In [ ]:
regressionWithOutliers :: (MonadSample m, MonadCond m) =>
    [Double] -> [Double] -> m (Double, Double, Double, Double, [Bool])
regressionWithOutliers xs ys = do
    slope <- normal 0 2
    intercept <- normal 0 2
    noise <- gamma 1 1
    probOutlier <- uniform 0 1 

    let observation (x, y) = do
        is_outlier <- bernoulli probOutlier
        let (mu, std) = if is_outlier
            then (0, 10)
            else (x*slope + intercept, noise)
        factor $ normalPdf mu std y
        return is_outlier
    
    outliers <- mapM observation $ zip xs ys
    return (slope, intercept, noise, probOutlier, outliers)

In [ ]:
mhRuns <- sampleIO $ prior $ mh 10000 $ regressionWithOutliers range (fst <$> samples)



In [ ]:
getOutliers (_,_,_,_,o) = o

os = getOutliers <$> mhRuns

bar :: [[Bool]] -> [(Int, Int)]
bar foo = foldr 
    (\lb li -> 
        [ if b then (num1+1, num2) else (num1,num2+1) | (b,(num1, num2)) <- zip lb li]) 
    (Prelude.repeat (0,0))
    foo



Line 6: Eta reduce
Found:
bar foo
  = foldr
      (\ lb li
         -> [if b then (num1 + 1, num2) else (num1, num2 + 1) |
               (b, (num1, num2)) <- zip lb li])
      (Prelude.repeat (0, 0)) foo
Why not:
bar
  = foldr
      (\ lb li
         -> [if b then (num1 + 1, num2) else (num1, num2 + 1) |
               (b, (num1, num2)) <- zip lb li])
      (Prelude.repeat (0, 0))

In [ ]:
predData = baseData . 
  dataColumn "Outlier Prediction" 
      -- (Strings (T.pack . show <$> head os) ) -- 
      (Numbers ((\(x, y) -> log (fromIntegral y / fromIntegral x)) <$> take 1000 (bar os)))
  


predEncoding = baseEncoding . color [ MName "Outlier Prediction", VL.MmType VL.Quantitative]



showPlot predEncoding predData

In [17]:
import qualified Data.Vector as VV
import Data.Fixed
import Control.Monad
import Control.Monad.Except
-- import Control.Monad

-- | The empirical distribution of a set of weighted samples
empirical :: Ord a => [(a, Double)] -> Either T.Text [(a, Double)]
empirical samples = runExcept $ do
    let (support, probs) = unzip samples
    when (any (<= 0) probs) (throwError "Probabilities are not all strictly positive")
    return $ enumerate $ do
      i <- categorical $ VV.fromList probs
      return $ support !! i


type Bin = (Double, Double)

toBin :: Double -- ^ bin size 
  -> Double -- ^ number
  -> Bin
toBin binSize n = let lb = n `mod'` binSize in (n-lb, n-lb + binSize) 

In [18]:
:e TupleSections
samples <- sampleIO $ prior $ mh 1000 $ toBin 0.01 <$> normal 0 1
s = empirical ((,1) <$> samples)

In [ ]:
-- s' = case s of
--     Right x -> x

-- x = fst . fst <$> s'
-- y = snd <$> s'

In [23]:
barPlot x y = 


  let enc = encoding
                    . position X [ PName "X", PmType Quantitative] -- , PBin [MaxBins 30]] --, PBin [AlreadyBinned True, MinStep 1.0]]
                    . position Y [ PName "Y", PmType Quantitative] --PAggregate Count, PmType Quantitative ]
                    -- . color cluster
                    -- . shape cluster

-- scaleOpts = [ SType ScLog, SDomain (DNumbers [3.5, 32]), SNice (IsNice False) ]

-- dat = (VL.VLData, either (error . show) id $ A.eitherDecode "['x',  'y']")

      dat = (dataFromColumns [ ] 
        . dataColumn "X" ((Numbers (x)))
        . dataColumn "Y" (Numbers y)
        -- . dataColumn "Year" (Strings [ "2010", "2014", "2015" ])
        ) []

  in toVegaLite [ 
              dat,
              mark Bar []
              , enc []
              , width 400
              , height 400
              ]



Line 15: Redundant bracket
Found:
((Numbers (x)))
Why not:
(Numbers (x))Line 15: Redundant bracket
Found:
(x)
Why not:
x

In [19]:
vlShow $ barPlot x y

: 

In [20]:
import Control.Monad.Trans.Cont
    ( cont, runCont, Cont, ContT(ContT) )
import Control.Monad.Bayes.Class (MonadSample (random, bernoulli, normal, uniformD, categorical, poisson, beta), condition, MonadInfer, factor)
import Statistics.Distribution (density)
import Numeric.Integration.TanhSinh
    ( trap, Result(result), absolute )
import Control.Monad.Bayes.Weighted (runWeighted, Weighted, prior)
import qualified Statistics.Distribution.Uniform as Statistics
import Numeric.Log (Log(ln, Exp))
import Data.Set (Set, fromList, elems)
import qualified Control.Monad.Bayes.Enumerator
import Control.Monad (replicateM, when)
import qualified Data.Vector as VV
import Control.Monad.Error (MonadError(throwError))
import Control.Monad.Except (runExcept)
import qualified Control.Monad.Bayes.Enumerator as Enumerator
import Control.Monad.Bayes.Sampler (sampleIO)
import Control.Monad.Bayes.Traced (mh)

newtype Measure a = Measure (Cont Double a) deriving (Functor, Applicative, Monad)

runMeasure :: (a -> Double) -> Measure a -> Double
runMeasure f (Measure a) = runCont a f

instance MonadSample Measure where
    random = fromDensityFunction $ density $ Statistics.uniformDistr 0 1
    bernoulli p = Measure $ cont (\f -> p * f True + (1 -p) * f False)

fromDensityFunction :: (Double -> Double) -> Measure Double
fromDensityFunction d = Measure $ cont $ \f ->
    quadratureTanhSinh (\x -> f x * d x)
  where
    quadratureTanhSinh = result . last . (\z -> trap z 0 1)

probability :: Ord a => (a, a) -> Weighted Measure a -> Double
probability (lower, upper) = runMeasure (\(x,d) -> if x<upper && x  > lower then exp $ ln d else 0) . runWeighted



In [ ]:
bins = toBin 0.1 <$> ([-3,-2.9..3] :: [Double])

xs = fst <$> bins

model = do
    x <- normal 0 1
    y <- normal 1 1
    return (x+y)

ys = [probability x $ model | x <- bins ]

vlShow $ barPlot xs ys
xs

-- probability (-0.2,0.1) $ normal 0 1

Line 10: Redundant $
Found:
probability x $ model
Why not:
probability x model

[-3.0,-2.9000000000000004,-2.8000000000000003,-2.7,-2.6,-2.5,-2.4000000000000004,-2.3000000000000003,-2.2,-2.1,-2.0,-1.9000000000000001,-1.8,-1.7000000000000002,-1.6,-1.5,-1.4000000000000001,-1.3,-1.2000000000000002,-1.1,-1.0,-0.9,-0.8,-0.7000000000000001,-0.6000000000000001,-0.5,-0.4,-0.30000000000000004,-0.2,-0.1,0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0]

In [27]:
bins = toBin 0.1 <$> ([-3,-2.9..3] :: [Double])

xs = fst <$> bins


model = do
    x <- beta 1 1
    y <- beta 1 1
    return (x `mod'` y)
    -- fromIntegral <$> poisson 2
    -- x <- poisson 3
    -- y <- beta 1 1
    -- return (fromIntegral x +  y)

ys = [probability x $ model | x <- bins ]

vlShow $ barPlot xs ys
xs

Line 15: Redundant $
Found:
probability x $ model
Why not:
probability x model

[-3.0,-2.9000000000000004,-2.8000000000000003,-2.7,-2.6,-2.5,-2.4000000000000004,-2.3000000000000003,-2.2,-2.1,-2.0,-1.9000000000000001,-1.8,-1.7000000000000002,-1.6,-1.5,-1.4000000000000001,-1.3,-1.2000000000000002,-1.1,-1.0,-0.9,-0.8,-0.7000000000000001,-0.6000000000000001,-0.5,-0.4,-0.30000000000000004,-0.2,-0.1,0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0]

In [37]:
bins = toBin 0.01 <$> ([0,0.01..1] :: [Double])

xs = fst <$> bins


model = do
    x <- beta 2 2
    -- y <- beta 1 1
    return (sin x * cos x)
    -- fromIntegral <$> poisson 2
    -- x <- poisson 3
    -- y <- beta 1 1
    -- return (fromIntegral x +  y)

ys = [probability x $ model | x <- bins ]

vlShow $ barPlot xs ys
xs

Line 15: Redundant $
Found:
probability x $ model
Why not:
probability x model

[0.0,1.0e-2,2.0e-2,2.0e-2,4.0e-2,5.0e-2,5.0e-2,7.0e-2,8.0e-2,8.0e-2,0.1,0.1,0.11,0.13,0.14,0.14,0.16,0.17,0.17,0.18,0.2,0.2,0.21,0.23,0.23,0.24,0.26,0.27,0.28,0.28,0.29,0.3,0.32,0.33,0.34,0.35000000000000003,0.35000000000000003,0.36,0.37,0.39,0.4,0.41000000000000003,0.41000000000000003,0.42,0.43,0.45,0.46,0.47000000000000003,0.47000000000000003,0.48,0.49,0.5,0.52,0.53,0.54,0.55,0.56,0.5700000000000001,0.5700000000000001,0.58,0.59,0.6,0.61,0.62,0.64,0.65,0.66,0.67,0.68,0.6900000000000001,0.7000000000000001,0.7000000000000001,0.71,0.72,0.73,0.74,0.75,0.77,0.78,0.79,0.8,0.81,0.8200000000000001,0.8300000000000001,0.8300000000000001,0.84,0.85,0.86,0.87,0.88,0.9,0.91,0.92,0.93,0.9400000000000001,0.9500000000000001,0.9500000000000001,0.96,0.97,0.98,0.99]